In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import optuna
import torchvision




import torch.nn.functional as F


class DenseNet_CNN_Model(nn.Module):
    def __init__(self, num_features, densenet_output_size, num_classes):
        super(DenseNet_CNN_Model, self).__init__()

        self.cnn_input_size = int(np.ceil(np.sqrt(num_features)))

        self.densenet = torchvision.models.densenet121(pretrained=True)
        self.densenet.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, densenet_output_size)
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        cnn_output_size = 32 * ((self.cnn_input_size // 2) // 2) ** 2

        self.fc = nn.Linear(densenet_output_size + cnn_output_size, num_classes)

    def forward(self, x):
        x_padded = F.pad(input=x, pad=(0, self.cnn_input_size**2 - x.size(1)), mode='constant', value=0)

        x_densenet = x_padded.view(x.size(0), 1, self.cnn_input_size, self.cnn_input_size)
        x_cnn = x_padded.view(x.size(0), 1, self.cnn_input_size, self.cnn_input_size)

        x_densenet = self.densenet(x_densenet)

        x_cnn = F.relu(self.conv1(x_cnn))
        x_cnn = self.pool(x_cnn)
        x_cnn = F.relu(self.conv2(x_cnn))
        x_cnn = self.pool(x_cnn)
        x_cnn = x_cnn.view(x_cnn.size(0), -1) 

        x_combined = torch.cat((x_densenet, x_cnn), dim=1)

        x_combined = self.fc(x_combined)
        return x_combined




# a helper class to load data, keep it as is
class MyDataset(data.Dataset):
    def __init__(self, x, y, device):
        self.x = torch.from_numpy(x).to(torch.float32)
        self.y = torch.from_numpy(y).to(torch.long) 
        self.device = device

    def __getitem__(self, index):
        xi = self.x[index].to(self.device)
        yi = self.y[index].to(self.device)
        return xi, yi

    def __len__(self):
        return len(self.y)


def seed_worker(worker_id):
    """
    This function is to ensure reproducibility

    """
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)



train_x = np.load('C:/Users/Tianrui Ye/Desktop/Final/train_x.npy', allow_pickle=True)
train_y = np.load('C:/Users/Tianrui Ye/Desktop/Final/train_y.npy', allow_pickle=True)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

torch.manual_seed(0)
 
dataset = MyDataset(train_x, train_y, torch.device("cpu:0"))
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, 
    worker_init_fn=seed_worker) 

model = DenseNet_CNN_Model(num_features=28169, densenet_output_size=64, num_classes=3)



criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

for epoch in range(50):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

    epoch_loss = running_loss / len(data_loader)
    epoch_acc = correct / total
    print(f'End of Epoch {epoch + 1}, Training Loss: {epoch_loss}, Training Accuracy: {epoch_acc}')


torch.save(model.state_dict(), './model.pth')

TypeError: LSTM_Model.__init__() got an unexpected keyword argument 'num_features'

In [13]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report
import torch
from torch.utils.data import TensorDataset, DataLoader

val_x_tensor = torch.from_numpy(val_x).to(torch.float32)
val_y_tensor = torch.from_numpy(val_y).to(torch.long)

val_dataset = TensorDataset(val_x_tensor, val_y_tensor)

val_loader = DataLoader(val_dataset, batch_size=32)

def evaluate_model(model, data_loader):
    model.eval() 
    predictions = []
    actuals = []
    with torch.no_grad(): 
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.tolist())
            actuals.extend(labels.tolist())

    accuracy = accuracy_score(actuals, predictions)
    conf_matrix = confusion_matrix(actuals, predictions)
    f1 = f1_score(actuals, predictions, average='weighted') 
    report = classification_report(actuals, predictions)

    return accuracy, conf_matrix, f1, report

accuracy, conf_matrix, f1, report = evaluate_model(model, val_loader)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"F1 Score: {f1}")
print(f"Classification Report:\n{report}")



Accuracy: 0.6129807692307693
Confusion Matrix:
[[ 67  56   3]
 [ 41 106  30]
 [  9  22  82]]
F1 Score: 0.6122759172447538
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.53      0.55       126
           1       0.58      0.60      0.59       177
           2       0.71      0.73      0.72       113

    accuracy                           0.61       416
   macro avg       0.62      0.62      0.62       416
weighted avg       0.61      0.61      0.61       416



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import optuna

torch.set_default_dtype(torch.float32)
class ImprovedLinear(nn.Module):
    def __init__(self, in_dim, 
                 n_hidden_1, n_hidden_2, n_hidden_3,
                 out_dim, dropout_p=0.5):
        super().__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.bn1 = nn.BatchNorm1d(n_hidden_1)  # 添加批量归一化
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.bn2 = nn.BatchNorm1d(n_hidden_2)  # 添加批量归一化
        self.layer3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.bn3 = nn.BatchNorm1d(n_hidden_3)  # 添加批量归一化
        self.layer4 = nn.Linear(n_hidden_3, out_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer4(x)
        return x


# a helper class to load data, keep it as is
class MyDataset(data.Dataset):
    def __init__(self, x, y, device):
        self.x = torch.from_numpy(x).to(torch.float32)
        self.y = torch.from_numpy(y).to(torch.long)  # Convert labels to Long type
        self.device = device

    def __getitem__(self, index):
        xi = self.x[index].to(self.device)
        yi = self.y[index].to(self.device)
        return xi, yi

    def __len__(self):
        return len(self.y)


def seed_worker(worker_id):
    """
    This function is to ensure reproducibility

    """
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    n_hidden_1 = trial.suggest_int("n_hidden_1", 100, 500)
    n_hidden_2 = trial.suggest_int("n_hidden_2", 100, 500)
    dropout_p = trial.suggest_float("dropout_p", 0.2, 0.5)

    train_x = np.load('C:/Users/Tianrui Ye/Desktop/Final/train_x.npy', allow_pickle=True)
    train_y = np.load('C:/Users/Tianrui Ye/Desktop/Final/train_y.npy', allow_pickle=True)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

    train_dataset = MyDataset(train_x, train_y, torch.device("cpu"))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker)

    val_dataset = MyDataset(val_x, val_y, torch.device("cpu"))
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = ImprovedLinear(train_x.shape[1], n_hidden_1, n_hidden_2, 3, dropout_p)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(10):
        model.train()
        for batch in train_loader:
            inputs, labels = batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    return avg_loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50) 

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-11-22 16:02:58,724] A new study created in memory with name: no-name-f6f65961-35c9-487c-9172-dc4c58fdd985
[W 2023-11-22 16:02:59,028] Trial 0 failed with parameters: {'lr': 0.0544601856276853, 'batch_size': 16, 'n_hidden_1': 454, 'n_hidden_2': 403, 'dropout_p': 0.42424439269086356} because of the following error: TypeError('empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:\n * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n').
Traceback (most recent call last):
  File "C:\Users\Tianrui Ye\AppData\Roaming\Python\Python311\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values =

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [9]:
import random
from nltk.tokenize import word_tokenize
# bi-gram model
def get_counts(context_length, training_text): # context_length:2  training_text: "This is a sample training text. This is used to demonstrate the text generation."
    counts = {}
    tokens = word_tokenize(training_text)

    for i in range(len(tokens) - context_length): # i: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
        context = tokens[i:i + context_length] # 0:2 -> 0,1
        next_token = tokens[i + context_length]
        
        context_tuple = tuple(context)
        if context_tuple in counts:
            if next_token in counts[context_tuple]:
                counts[context_tuple][next_token] += 1
            else:
                counts[context_tuple][next_token] = 1 # "This is" -> {"a": 1, "used": 1}
        else:
            counts[context_tuple] = {next_token: 1} # key -> value
    
    return counts

def generate_from_file(context_length, training_text, output_length=10):
    counts = get_counts(context_length, training_text)

    first_tokens = random.choice(list(counts.keys())) # "This is"
    output_list = list(first_tokens) # ["This", "is"]
    current_context = first_tokens # ("This", "is") -> "a"    ("is","a") -> "sample"

    for i in range(output_length): # 0~10
        if current_context in counts and counts[current_context] is not None:
            next_token = max(counts[current_context], key=counts[current_context].get)
            temp = list(current_context[1:]) + [next_token]
            current_context = tuple(temp)
            output_list.append(next_token)

    return " ".join(output_list)

# Example usage
training_text = "This is a sample training text. This is used to demonstrate the text generation."
output = generate_from_file(2, training_text, 15)
print(output)


{('This', 'is'): {'a': 1, 'used': 1}, ('is', 'a'): {'sample': 1}, ('a', 'sample'): {'training': 1}, ('sample', 'training'): {'text': 1}, ('training', 'text'): {'.': 1}, ('text', '.'): {'This': 1}, ('.', 'This'): {'is': 1}, ('is', 'used'): {'to': 1}, ('used', 'to'): {'demonstrate': 1}, ('to', 'demonstrate'): {'the': 1}, ('demonstrate', 'the'): {'text': 1}, ('the', 'text'): {'generation': 1}, ('text', 'generation'): {'.': 1}}
['training', 'text']
training text . This is a sample training text . This is a sample training text .
